In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical

In [2]:
from sklearn import preprocessing
import pandas as pd

# This function returns a normalized version of the input dataframe 
def normalize(df):
    normalized_df = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        normalized_df[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
        
    return normalized_df

In [120]:
### Get Dataset ###
def load_data_kfold(k):
    
    DATASET_PATH = "emotion-recognition-by-voice/datasets/dataset_48.csv"
    NUMBER_OF_SPLITS = 10
    df = pd.read_csv(DATASET_PATH, sep=",")

    X = df[df.columns[3:51]] # Only the MFCC features
    y = df[df.columns[-1]] # Emotion label

    # Normalization of input features in X
    X = normalize(X)

    # Split the dataset in train and test
#     X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
    X_train = X
    y_train = y
    


    folds = list(StratifiedKFold(n_splits=NUMBER_OF_SPLITS, shuffle=True, random_state=1).split(X_train, y_train))

    return folds, X_train, y_train

In [40]:
## Callback Functions ##

def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    return [mcp_save, reduce_lr_loss]


In [130]:
## Model ##
def get_model():
    model = Sequential()

    model.add(Conv1D(128, 5,padding='same', input_shape=(48,1), name="conv1"))
    model.add(Activation('sigmoid', name="activation1"))
    model.add(Conv1D(64, 5,padding='same', name="conv2"))
    model.add(Activation('sigmoid', name="activation2"))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(32, 5,padding='same',))
    model.add(Activation('sigmoid'))
    model.add(Conv1D(32, 5,padding='same',))
    model.add(Activation('sigmoid'))
    model.add(Flatten())
    model.add(Dense(7))
    model.add(Activation('softmax', name="last_activation"))
#     model.add(Flatten())
#     model.add(Dense(1))
#     model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [121]:
folds, X_train, y_train = load_data_kfold(7)
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [85]:
X_train_cv[0]
X_train
X_traincnn = np.expand_dims(X_train, axis=2)

In [109]:
y_train[0]

array([0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [78]:

model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_89 (Conv1D)           (None, 48, 128)           768       
_________________________________________________________________
activation_111 (Activation)  (None, 48, 128)           0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 48, 64)            41024     
_________________________________________________________________
activation_112 (Activation)  (None, 48, 64)            0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 48, 64)            0         
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 6, 64)             0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 6, 32)           

In [ ]:
######### K Fold ##############
batch_size = 12
EPOCHS = 100
    
for jindex , (train_idx, val_idx) in enumerate(folds):
    print('\nFold ',jindex)
#     print(train_idx)
#     print(val_idx)
    X_train_cv = X_train.values[train_idx]
    y_train_cv = y_train[train_idx]
    X_valid_cv = X_train.values[val_idx]
    y_valid_cv= y_train[val_idx]
    
    
    name_weights = "final_model_fold" + str(jindex) + "_weights.h5"
    callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
    model = get_model()
    
    X_traincnn = np.expand_dims(X_train_cv, axis=2)
    X_validcnn = np.expand_dims(X_valid_cv, axis=2)

    model.fit(  X_traincnn,
                y_train_cv,
                epochs=EPOCHS,
                shuffle=True,
                verbose=1,
                validation_data = (X_validcnn, y_valid_cv),
                callbacks = callbacks
             )


Fold  0


/usr/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 954 samples, validate on 112 samples
Epoch 1/100
954/954 [==============================] - 1s 1ms/step - loss: 1.9711 - accuracy: 0.1436 - val_loss: 1.9677 - val_accuracy: 0.1429
Epoch 2/100
954/954 [==============================] - 0s 449us/step - loss: 1.9657 - accuracy: 0.1205 - val_loss: 1.9535 - val_accuracy: 0.1429
Epoch 3/100
954/954 [==============================] - 1s 612us/step - loss: 1.9631 - accuracy: 0.1300 - val_loss: 1.9613 - val_accuracy: 0.1429
Epoch 4/100
954/954 [==============================] - 1s 571us/step - loss: 1.9581 - accuracy: 0.1394 - val_loss: 1.9618 - val_accuracy: 0.1429
Epoch 5/100
954/954 [==============================] - 0s 469us/step - loss: 1.9598 - accuracy: 0.1436 - val_loss: 1.9539 - val_accuracy: 0.1429
Epoch 6/100
954/954 [==============================] - 0s 448us/step - loss: 1.9605 - accuracy: 0.1373 - val_loss: 1.9558 - val_accuracy: 0.1429
Epoch 7/100
954/954 [==============================] - 1s 629us/step - loss: 1.9578 - 

Epoch 55/100
954/954 [==============================] - 1s 845us/step - loss: 1.9459 - accuracy: 0.1509 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 56/100
954/954 [==============================] - 1s 1ms/step - loss: 1.9459 - accuracy: 0.1384 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 57/100
954/954 [==============================] - 1s 859us/step - loss: 1.9459 - accuracy: 0.1436 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 58/100
954/954 [==============================] - 1s 968us/step - loss: 1.9459 - accuracy: 0.1247 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 59/100
954/954 [==============================] - 1s 962us/step - loss: 1.9459 - accuracy: 0.1384 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 60/100
954/954 [==============================] - 1s 706us/step - loss: 1.9459 - accuracy: 0.1447 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 61/100
954/954 [==============================] - 1s 696us/step - loss: 1.9459 - accuracy: 0.1352 - val_loss: 1.9460 - v

/usr/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 955 samples, validate on 111 samples
Epoch 1/100
955/955 [==============================] - 1s 924us/step - loss: 1.9887 - accuracy: 0.1131 - val_loss: 1.9523 - val_accuracy: 0.1441
Epoch 2/100
955/955 [==============================] - 1s 535us/step - loss: 1.9547 - accuracy: 0.1382 - val_loss: 1.9511 - val_accuracy: 0.1441
Epoch 3/100
955/955 [==============================] - 1s 716us/step - loss: 1.9632 - accuracy: 0.1382 - val_loss: 1.9533 - val_accuracy: 0.1441
Epoch 4/100
955/955 [==============================] - 1s 634us/step - loss: 1.9606 - accuracy: 0.1225 - val_loss: 1.9514 - val_accuracy: 0.1441
Epoch 5/100
955/955 [==============================] - 1s 686us/step - loss: 1.9530 - accuracy: 0.1424 - val_loss: 1.9524 - val_accuracy: 0.1441
Epoch 6/100
955/955 [==============================] - 1s 772us/step - loss: 1.9571 - accuracy: 0.1382 - val_loss: 1.9623 - val_accuracy: 0.1441
Epoch 7/100
955/955 [==============================] - 1s 716us/step - loss: 1.9625 

Epoch 57/100
955/955 [==============================] - 1s 699us/step - loss: 1.9474 - accuracy: 0.1267 - val_loss: 1.9463 - val_accuracy: 0.1441
Epoch 58/100
955/955 [==============================] - 0s 479us/step - loss: 1.9468 - accuracy: 0.1288 - val_loss: 1.9458 - val_accuracy: 0.1441
Epoch 59/100
955/955 [==============================] - 1s 682us/step - loss: 1.9472 - accuracy: 0.1267 - val_loss: 1.9459 - val_accuracy: 0.1441
Epoch 60/100
955/955 [==============================] - 1s 585us/step - loss: 1.9468 - accuracy: 0.1309 - val_loss: 1.9459 - val_accuracy: 0.1441
Epoch 61/100
955/955 [==============================] - 1s 564us/step - loss: 1.9472 - accuracy: 0.1257 - val_loss: 1.9458 - val_accuracy: 0.1441
Epoch 62/100
955/955 [==============================] - 1s 637us/step - loss: 1.9470 - accuracy: 0.1309 - val_loss: 1.9461 - val_accuracy: 0.1441
Epoch 63/100
955/955 [==============================] - 0s 444us/step - loss: 1.9470 - accuracy: 0.1288 - val_loss: 1.9460 -